In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import yfinance as yf

In [ ]:
n_steps = 252
n_paths = 100
T = 1
tickers = ["GOOGL", "NVDA", "FB",
           "PLTR", "WFC",
           "DIS", "LUV", "PFE",
           "COKE", "CAT"]
weights = [0.4154753,  0.08737277, 0.02717975, 0.18494651, 0.03159597, 0.00579978, 0.0066295, 0.12686745, 0.03158073, 0.08255224]


In [ ]:
data = yf.download(tickers, start="2025-01-01", end="2026-01-01")["Close"]

In [ ]:
S0 = data.values[-1]

log_returns = np.log(data / data.shift(1)).dropna()

mu = log_returns.mean().values
sigma = log_returns.std().values
corr = data.corr()

In [ ]:
def gbm_paths(
    S0: list, mu: list, sigma: list, corr:
    np.ndarray, T: float, N: int, n_paths: int
  ):
  S0 = np.asarray(S0)
  mu = np.asarray(mu)
  sigma = np.asarray(sigma)

  n_assets = len(S0)
  dt = T/N

  eps = np.finfo(float).eps
  eigvals, eigvec = np.linalg.eigh(corr)
  eigvals = np.clip(eigvals, eps, None)
  corr_pd = eigvec @ np.diag(eigvals) @ eigvec.T

  L = np.linalg.cholesky(corr_pd)
  Z = np.random.standard_normal((n_paths, N, n_assets))
  Z_corr = Z @ L.T

  W = np.cumsum(np.sqrt(dt) * Z_corr, axis=1)

  t = np.linspace(dt, T, N).reshape(1, N, 1)

  paths = S0 * np.exp(
      (mu - 0.5 * sigma**2) * t + sigma * W
  )
  return paths

In [ ]:
paths = gbm_paths(S0, mu, sigma, corr=corr, T=1, N=n_steps, n_paths=n_paths)

In [ ]:
weights = np.array(weights)
weighted_paths = paths.sum(axis=2)
plt.plot(weighted_paths.T, label="Sample paths")
plt.show()
